In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import regex as re

In [4]:
db_path = "sqlite:///spotify.db"
engine = create_engine(db_path)

First, we decided which columns we actually wanted to work with. The year released column was ambiguous (some songs were released after the year they were popular(possibly a descrepancy between song release and album release if the song was released earlier as a single)) as was therefore dropped The date the song was added to this database is not neccessary for our analysis We were not interested in the live, val, acous, or pop columns so they were dropped as well

In [5]:
df_messy = pd.read_csv("spotify_data.csv")

In [7]:
df_messy.head()
df = df_messy[['title', 'artist', 'top genre', 'bpm', 'nrgy',
       'dnce', 'dB', 'dur', 'spch', 'top year',
       'artist type']]
df['title_id'] = df.index

C:\Users\OGUNSAKINS\AppData\Local\Temp\ipykernel_6232\199577109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title_id'] = df.index


In [8]:
df

,title,artist,top genre,bpm,nrgy,dnce,dB,dur,spch,top year,artist type,title_id
0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,140.0,81.0,61.0,-6.0,203.0,6.0,2010.0,Duo,0
1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,138.0,89.0,68.0,-4.0,192.0,8.0,2010.0,Duo,1
2,I Need A Dollar,Aloe Blacc,pop soul,95.0,48.0,84.0,-7.0,243.0,3.0,2010.0,Solo,2
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,93.0,87.0,66.0,-4.0,180.0,12.0,2010.0,Solo,3
4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,104.0,85.0,69.0,-6.0,268.0,5.0,2010.0,Solo,4
...,...,...,...,...,...,...,...,...,...,...,...,...
998,Strike a Pose (feat. Aitch),Young T & Bugsey,afroswing,138.0,58.0,53.0,-6.0,214.0,10.0,2019.0,Duo,998
999,The London (feat. J. Cole & Travis Scott),Young Thug,atl hip hop,98.0,59.0,80.0,-7.0,200.0,15.0,2019.0,Solo,999
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001


Many of the songs have featuring artists so a column was created for them. It splits the titles into the actual title and the artist. There were several different ways featuring artists were recognized, and separate cases were created to catch them all.

In [9]:
feat = []
titles = []
print(type(df['title'].loc[1]))
for i in range(len(df)):
    full_artist = str(df['title'].loc[i])
    title = full_artist
    if "(feat. " in full_artist:
        artist = full_artist.split("(feat. ")[1] #artist is listed 2nd
        title = full_artist.split("(feat. ")[0] #this is the actual title
        artist = artist[0:-1] #leave off trailing )
        if ", " in artist:
            artist = re.split(",|&", artist)#split on wither a comma or ampersand
        if "& " in artist:
            artist = re.split(",|&", artist)
        feat.append(artist)
        titles.append(title)
    elif "(Featuring " in full_artist: #this happens twice rows85,86
        artist = full_artist.split("(Featuring ")[1]
        title = full_artist.split("(Featuring ")[0]
        artist = artist[0:-1]
        if ", " in artist:
            artist = re.split(",|&", artist)
        if "& " in artist:
            artist = re.split(",|&", artist)
        feat.append(artist)
        titles.append(title)
    elif "vs. " in full_artist: #this only happens in one case row74
        artist = full_artist.split(" - ")[1]
        title = full_artist.split(" - ")[0]
        artist2 = artist.split(" vs. ")[1]
        feat.append(artist2)
        titles.append(title)
    elif "Vocal Edit" in full_artist: #this happens twice row 84 and 38
        if '(' in full_artist:  
            artist = full_artist.split("(")[1]
            title = full_artist.split("(")[0]
        elif '-' in full_artist:
            artist = full_artist.split(' - ')[1]
            title = full_artist.split(' - ')[0]
        artist2 = artist.split(" ")[0] #only want artist name
        feat.append(artist2)
        titles.append(title)
    elif "(with " in full_artist:
        artist = full_artist.split("(with ")[1]
        title = full_artist.split("(with ")[0]
        artist = artist[0:-1]
        if ", " in artist:
            artist = re.split(",|&", artist)
        if "& " in artist:
            artist = re.split(",|&", artist)
        feat.append(artist)
        titles.append(title)
    else:
        feat.append(np.nan) #if there is no featured artist, fill in space with nan
        titles.append(title)


df['feat'] = pd.Series(feat)
df['title'] = pd.Series(titles)
feat_df = pd.DataFrame(feat)

<class 'str'>


C:\Users\OGUNSAKINS\AppData\Local\Temp\ipykernel_6232\3895773808.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['feat'] = pd.Series(feat)
C:\Users\OGUNSAKINS\AppData\Local\Temp\ipykernel_6232\3895773808.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = pd.Series(titles)


feat_df is destined to be a relationship table, but it needs time

In [10]:
feat_df['title_id'] = feat_df.index
feat_df = feat_df.rename(columns={0:'feat_artist'})
feat_df = feat_df.dropna(subset='feat_artist').reset_index()
feat_df = feat_df.explode('feat_artist').reset_index()
feat_df = feat_df[['feat_artist', 'title_id']]
feat_df

,feat_artist,title_id
0,Katy Perry,0
1,Ke$ha,1
2,Hayley Williams of Paramore,3
3,Bruno Mars,4
4,Rivers Cuomo,5
...,...,...
240,RANI,985
241,Idris Elba,997
242,Aitch,998
243,J. Cole,999


feat_artist_df is just a table with the featured artist name and id

In [11]:
feat_artist_df = pd.DataFrame(feat_df['feat_artist'].unique()).reset_index()
feat_artist_df['feat_artist_id'] = feat_artist_df.index
feat_artist_df = feat_artist_df.rename(columns={0:"feat_artist"})
feat_artist_df = feat_artist_df[['feat_artist', 'feat_artist_id']]
feat_artist_df

,feat_artist,feat_artist_id
0,Katy Perry,0
1,Ke$ha,1
2,Hayley Williams of Paramore,2
3,Bruno Mars,3
4,Rivers Cuomo,4
...,...,...
183,RANI,183
184,Idris Elba,184
185,Aitch,185
186,J. Cole,186


NOW feat_df is a relationship table. With a composite key made of feat_artist_id and title_id. title_id is the index of df.

In [12]:
feat_df = feat_df.merge(feat_artist_df, left_on='feat_artist', right_on='feat_artist', how='inner')
feat_df = feat_df[['feat_artist_id', 'title_id']]
feat_df.set_index(['feat_artist_id', 'title_id'], inplace=True)
feat_df

Empty DataFrame
Columns: []
Index: [(0, 0), (0, 86), (1, 1), (1, 475), (2, 3), (3, 4), (3, 90), (3, 285), (3, 548), (4, 5), (5, 12), (6, 12), (7, 13), (8, 14), (9, 14), (10, 15), (10, 181), (11, 15), (12, 15), (13, 15), (13, 417), (14, 27), (15, 45), (16, 74), (16, 159), (16, 279), (16, 322), (17, 84), (18, 85), (19, 88), (20, 91), (20, 222), (20, 796), (21, 92), (22, 94), (23, 95), (24, 97), (24, 197), (25, 97), (25, 197), (26, 116), (27, 118), (27, 445), (27, 857), (28, 119), (29, 119), (30, 123), (31, 126), (31, 967), (32, 126), (33, 127), (33, 614), (34, 143), (35, 177), (36, 180), (36, 214), (37, 180), (38, 180), (39, 215), (40, 233), (41, 237), (42, 238), (42, 341), (43, 256), (44, 267), (45, 282), (45, 413), (46, 283), (47, 284), (48, 301), (48, 810), (48, 913), (48, 966), (49, 301), (50, 301), (51, 319), (52, 320), (52, 510), (53, 321), (54, 328), (55, 328), (56, 329), (56, 714), (57, 330), (58, 330), (59, 335), (60, 344), (61, 345), (61, 437), (61, 898), (62, 351), (62, 410), (63, 362), (64, 363), (65, 364), (66, 375), (67, 379), (68, 380), (69, 382), (70, 385), ...]

[245 rows x 0 columns]

artist_df is a table with the artist names. This was created bc artist type depended on artist and not on the title.

In [13]:
artist_df = df[['artist', "artist type"]]
artist_df = artist_df.drop_duplicates(subset='artist').reset_index()
artist_df['artist_id'] = artist_df.index
artist_df = artist_df[['artist_id','artist', 'artist type']]

nums_df has all of the number columns. This was created to clean up the clutter of the main table. the index is the same as the index of songs_df

In [14]:
nums_df = df[['title_id','bpm', 'nrgy','dnce', 'dB', 'dur', 'spch' ]]
nums_df

,title_id,bpm,nrgy,dnce,dB,dur,spch
0,0,140.0,81.0,61.0,-6.0,203.0,6.0
1,1,138.0,89.0,68.0,-4.0,192.0,8.0
2,2,95.0,48.0,84.0,-7.0,243.0,3.0
3,3,93.0,87.0,66.0,-4.0,180.0,12.0
4,4,104.0,85.0,69.0,-6.0,268.0,5.0
...,...,...,...,...,...,...,...
998,998,138.0,58.0,53.0,-6.0,214.0,10.0
999,999,98.0,59.0,80.0,-7.0,200.0,15.0
1000,1000,NaN,NaN,NaN,NaN,NaN,NaN
1001,1001,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
song_df = df[['title_id','title','artist' ,'top genre', 'top year']]
song_df = song_df.merge(artist_df, left_on="artist", right_on='artist', how='inner') 
song_df = song_df[['title_id','title', 'top genre', 'top year', 'artist_id']]
song_df

,title_id,title,top genre,top year,artist_id
0,0,STARSTRUKK,dance pop,2010.0,0
1,1,My First Kiss,dance pop,2010.0,0
2,2,I Need A Dollar,pop soul,2010.0,1
3,403,The Man,pop soul,2014.0,1
4,3,Airplanes,atl hip hop,2010.0,2
...,...,...,...,...,...
998,998,Strike a Pose,afroswing,2019.0,442
999,999,The London,atl hip hop,2019.0,443
1000,1000,nan,NaN,NaN,444
1001,1001,nan,NaN,NaN,444


In [16]:
artist_df.to_sql(name="artist_df",con=engine, if_exists='replace')
nums_df.to_sql(name="nums_df",con=engine, if_exists='replace')
song_df.to_sql(name="song_df",con=engine, if_exists='replace')
feat_df.to_sql(name='feat_df', con=engine, if_exists='replace')
feat_artist_df.to_sql(name='feat_artist_df', con=engine, if_exists='replace')

188